Importy

In [60]:
import os
import pandas as pd

Wczytywanie danych

In [61]:
data_path = "./data"
file_list = os.listdir(data_path)
func = lambda  a : a.replace(".txt", "").split("-")
count_size = lambda a : os.stat(a).st_size
data = [[func(title)[0], func(title)[1], data_path + title, count_size(data_path + title)] for title in file_list]
df = pd.DataFrame(data, columns=["Author", "Name of The work", "FilePath", "FileSize"])

FileNotFoundError: [WinError 2] Nie można odnaleźć określonego pliku: './dataAdamMickiewicz-Burza.txt'

Przedstawiam zestaw uczący. Mamy tutaj pozycję kilku Polskich autorów.

In [ ]:
print(df.drop(columns=["Name of The work", "FilePath"]).groupby("Author").sum())

                   FileSize
Author                     
AdamMickiewicz       516084
HenrykSienkiewicz    867390
              Author    Name of The work  \
0     AdamMickiewicz               Burza   
1     AdamMickiewicz              Dziady   
2     AdamMickiewicz          PanTadeusz   
3  HenrykSienkiewicz  OgniemIMieczemTom1   

                                        FilePath  FileSize  
0                  data/AdamMickiewicz-Burza.txt       742  
1                 data/AdamMickiewicz-Dziady.txt     23305  
2             data/AdamMickiewicz-PanTadeusz.txt    492037  
3  data/HenrykSienkiewicz-OgniemIMieczemTom1.txt    867390  


Implementacje tablicy mieszającej pobraliśmy ze źródła: https://www.geeksforgeeks.org/hash-map-in-python/

In [ ]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.hash_table = [[] for _ in range(self.size)]

    def put(self, key, val):
        hashed_key = hash(key) % self.size
        bucket = self.hash_table[hashed_key]
        found_key = False
        for index, record in enumerate(bucket):
            record_key, record_val = record
            if record_key == key:
                found_key = True
                break
        if found_key:
            bucket[index] = (key, val)
        else:
            bucket.append((key, val))
            
    def get(self, key):
        hashed_key = hash(key) % self.size
        bucket = self.hash_table[hashed_key]
        found_key = False
        for index, record in enumerate(bucket):
            record_key, record_val = record
            if record_key == key:
                found_key = True
                break
        if found_key:
            return record_val
        else:
            return None

    def pop(self, key):
        hashed_key = hash(key) % self.size
        bucket = self.hash_table[hashed_key]
        found_key = False
        for index, record in enumerate(bucket):
            record_key, record_val = record

            if record_key == key:
                found_key = True
                break
        if found_key:
            bucket.pop(index)
            return record_val
        return None

Wczytanie stop-słów:

In [ ]:
delimeters = ["\\n", "'"]
stop_words = HashTable(350)
with open("./stopyPL.txt", encoding="utf8") as f:
    for line in f:
        word = repr(line)
        for c in delimeters:
            word = word.replace(c, "")
        if word:
            stop_words.put(word, word)

Projekt ten używa worka słów, który to przechowuje informacje na temat częstości występowania słów w danym dokumencie. Definiujemy tu dwie funckje jedna do standaryzacji słów tzn. usunięcia ewentualnych znaków graficznych takich jak kropka czy przecinek oraz ustawienie wielkości liter na małe aby słowa "Który" oraz "który" były zliczane jako to same słowo.
Druga funkcja służy do budowania worka słów. Zlicza najpierw pojedyńcze przypadki wystąpień, a następnie zlicza częstość wystąpień danego słowa.

In [ ]:
def normalize_word(word):
    word = word.lower()
    special_char = [".", ",", "-", "?", "(", ")", "!", "\\", "\"", ":", ";", "*"]
    for char in special_char:
        word = word.replace(char, "")
    return word

def remove_stop_words(word):
    pass

def generate_word_bag(FileName, rmstops=True):
    word_bag = HashTable(5000)
    over_all_words = 0
    every_word = []
    with open(FileName, "r", encoding="utf8") as f:
        for line in f:
            words = line.split()
            words = [normalize_word(word) for word in words]
            for word in words:
                if word != '':
                    if rmstops and stop_words.get(word) is not None:
                        continue
                    over_all_words += 1
                    counter = word_bag.pop(word)
                    if counter is not None:
                        word_bag.put(word, counter + 1)
                        every_word.append(word)
                    else:
                        word_bag.put(word, 1)
        f.close()
    for word in every_word:
        counter = word_bag.pop(word)
        word_bag.put(word, counter/over_all_words)
    return word_bag, every_word

In [66]:
def head(words, dictionary, start=0, end=None, step=1, fname=None):
    if end is None:
        end = min(5, len(words))
    if fname is not None:
        print("W pliku: \"" + fname + "\"")
    for word in words[start:end:step]:
        print("Słowo: \"", word, "\" występowało z częstością: ", dictionary.get(word), sep="")

In [68]:
file = df["FilePath"][1]
dictionary, words = generate_word_bag(file)
head(words, dictionary, end=30, fname="data/AdamMickiewicz-Dziady.txt")

W pliku: "data/AdamMickiewicz-Dziady.txt"
Słowo: "dziady" występowało z częstością: 5.116493114424868e-23
Słowo: "dziady" występowało z częstością: 5.116493114424868e-23
Słowo: "dziady" występowało z częstością: 5.116493114424868e-23
Słowo: "zwyczaj" występowało z częstością: 0.0009718172983479105
Słowo: "zmarłych" występowało z częstością: 0.0009718172983479105
Słowo: "czasów" występowało z częstością: 0.0009718172983479105
Słowo: "dotąd" występowało z częstością: 4.589061522683342e-10
Słowo: "dziady" występowało z częstością: 5.116493114424868e-23
Słowo: "uroczystości" występowało z częstością: 0.0009718172983479105
Słowo: "pospólstwo" występowało z częstością: 0.0009718172983479105
Słowo: "obrzędy" występowało z częstością: 7.083216460261739e-07
Słowo: "niegdyś" występowało z częstością: 4.589061522683342e-10
Słowo: "pewne" występowało z częstością: 0.0009718172983479105
Słowo: "poema" występowało z częstością: 0.0009718172983479105
Słowo: "guślarz" występowało z częstością: 9.60074

Teraz musimy stworzyć model, który będzie klasyfikował do jakiego autora należy dany utwór.